In [22]:
pip install open3d numpy


Note: you may need to restart the kernel to use updated packages.


In [23]:
import open3d as o3d
import numpy as np
from tqdm import tqdm

# Step 1: Load & normalize mesh
mesh = o3d.geometry.TriangleMesh.create_sphere(radius=1.0)
mesh.compute_vertex_normals()

# Normalize: center and scale
center = mesh.get_center()
mesh.translate(-center)
bbox = mesh.get_axis_aligned_bounding_box()
scale = 1.0 / max(bbox.get_extent())
mesh.scale(scale, center=(0, 0, 0))

# Step 2: Sample 8 candidate viewpoints on a sphere
def fibonacci_sphere(samples=32, radius=2.5):
    points = []
    phi = np.pi * (3. - np.sqrt(5.))  # golden angle
    for i in range(samples):
        y = 1 - (i / float(samples - 1)) * 2
        r = np.sqrt(1 - y * y)
        theta = phi * i
        x = np.cos(theta) * r
        z = np.sin(theta) * r
        points.append([x * radius, y * radius, z * radius])
    return np.array(points)

candidate_views = fibonacci_sphere()

# Step 3: Sample mesh as point cloud (surface points)
pcd = mesh.sample_points_uniformly(number_of_points=5000)

# Step 4: Simulate visibility per viewpoint with progress bar
view_coverage = []
print("Simulating visibility from each viewpoint:")
for i, view in tqdm(enumerate(candidate_views), total=len(candidate_views)):
    _, pt_map = pcd.hidden_point_removal(view, radius=5.0)
    view_coverage.append(set(pt_map))

# Step 5: Print stats
total_points = set(range(len(pcd.points)))
print(f"\nTotal surface points: {len(total_points)}")
for i, indices in enumerate(view_coverage):
    print(f"View {i}: sees {len(indices)} points")


Simulating visibility from each viewpoint:


100%|█████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 119.29it/s]


Total surface points: 5000
View 0: sees 2159 points
View 1: sees 2099 points
View 2: sees 2090 points
View 3: sees 2042 points
View 4: sees 2078 points
View 5: sees 2092 points
View 6: sees 2012 points
View 7: sees 2089 points
View 8: sees 2079 points
View 9: sees 2055 points
View 10: sees 2096 points
View 11: sees 2011 points
View 12: sees 2064 points
View 13: sees 2084 points
View 14: sees 1992 points
View 15: sees 2107 points
View 16: sees 2041 points
View 17: sees 2022 points
View 18: sees 2094 points
View 19: sees 2011 points
View 20: sees 2065 points
View 21: sees 2058 points
View 22: sees 2007 points
View 23: sees 2087 points
View 24: sees 2032 points
View 25: sees 2068 points
View 26: sees 2052 points
View 27: sees 1995 points
View 28: sees 2081 points
View 29: sees 2018 points
View 30: sees 2059 points
View 31: sees 2091 points


In [24]:
!pip install mealpy


In [25]:
import numpy as np
from mealpy import FloatVar, SMA

# --- Configurations ---
num_views_to_select = 10
num_candidates = len(view_coverage)
total_points = set(range(len(pcd.points)))

# --- Fitness Function (minimize uncovered surface points) ---
def objective_function(solution):
    indices = [int(round(x)) % num_candidates for x in solution]
    indices = list(set(indices))  # Remove duplicates
    covered = set()
    for i in indices:
        covered |= view_coverage[i]
    uncovered = len(total_points - covered)
    return uncovered  # Return scalar

# --- Define problem_dict with FloatVar ---
problem_dict = {
    "obj_func": objective_function,
    "minmax": "min",
    "bounds": FloatVar(
        lb=(0,) * num_views_to_select,
        ub=(num_candidates - 1,) * num_views_to_select,
        name="view_index"
    )
}

# --- Run SMA ---
model = SMA.OriginalSMA(epoch=50, pop_size=20, p_t=0.03)
g_best = model.solve(problem_dict)

# --- Evaluate selected views ---
selected_indices = sorted(set(int(round(x)) % num_candidates for x in g_best.solution))
covered = set()
for i in selected_indices:
    covered |= view_coverage[i]
coverage_percent = (len(covered) / len(total_points)) * 100

# --- Print Results ---
print(f"\n📌 Selected viewpoints: {selected_indices}")
print(f"✅ Coverage: {coverage_percent:.2f}%")
print(f"❌ Uncovered points: {g_best.target.fitness}")


2025/07/14 08:34:36 PM, INFO, mealpy.bio_based.SMA.OriginalSMA: OriginalSMA(epoch=50, pop_size=20, p_t=0.03)
2025/07/14 08:34:36 PM, INFO, mealpy.bio_based.SMA.OriginalSMA: >>>Problem: P, Epoch: 1, Current best: 0.0, Global best: 0.0, Runtime: 0.03472 seconds
2025/07/14 08:34:36 PM, INFO, mealpy.bio_based.SMA.OriginalSMA: >>>Problem: P, Epoch: 2, Current best: 0.0, Global best: 0.0, Runtime: 0.02368 seconds
2025/07/14 08:34:36 PM, INFO, mealpy.bio_based.SMA.OriginalSMA: >>>Problem: P, Epoch: 3, Current best: 0.0, Global best: 0.0, Runtime: 0.02583 seconds
2025/07/14 08:34:36 PM, INFO, mealpy.bio_based.SMA.OriginalSMA: >>>Problem: P, Epoch: 4, Current best: 0.0, Global best: 0.0, Runtime: 0.02169 seconds
2025/07/14 08:34:36 PM, INFO, mealpy.bio_based.SMA.OriginalSMA: >>>Problem: P, Epoch: 5, Current best: 0.0, Global best: 0.0, Runtime: 0.02162 seconds
2025/07/14 08:34:36 PM, INFO, mealpy.bio_based.SMA.OriginalSMA: >>>Problem: P, Epoch: 6, Current best: 0.0, Global best: 0.0, Runtime: 0


📌 Selected viewpoints: [3, 5, 7, 15, 16, 19, 21, 24, 27]
✅ Coverage: 100.00%
❌ Uncovered points: 0.0


In [26]:
import open3d as o3d

# Ensure mesh is not modified in place
mesh_vis = mesh.translate((0, 0, 0), relative=False)

# Create viewpoint spheres
spheres = []
for i, pos in enumerate(candidate_views):
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.05)
    sphere.translate(pos)
    if i in selected_indices:
        sphere.paint_uniform_color([1.0, 0.0, 0.0])  # 🔴 Red = selected
    else:
        sphere.paint_uniform_color([0.7, 0.7, 0.7])  # ⚪ Light gray = unselected
    spheres.append(sphere)

# Visualize
o3d.visualization.draw_geometries([mesh_vis] + spheres)


In [27]:
import urllib.request
import zipfile
import os
from tqdm import tqdm

# URL and target paths
url = "https://modelnet.cs.princeton.edu/ModelNet40.zip"
zip_path = "ModelNet40.zip"
extract_path = "ModelNet40"

# Download with progress bar
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

print("📥 Downloading ModelNet40.zip...")
with DownloadProgressBar(unit='B', unit_scale=True, miniters=1, desc=zip_path) as t:
    urllib.request.urlretrieve(url, filename=zip_path, reporthook=t.update_to)

# Extract with progress bar
print("📦 Extracting ModelNet40.zip...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()
    for file in tqdm(file_list, desc="Extracting", unit="files"):
        zip_ref.extract(file, extract_path)

print("✅ Done! Files extracted to:", extract_path)


📥 Downloading ModelNet40.zip...


ModelNet40.zip: 2.04GB [07:34, 4.48MB/s]                                                                               


📦 Extracting ModelNet40.zip...


Extracting: 100%|████████████████████████████████████████████████████████████| 12432/12432 [00:40<00:00, 303.34files/s]

✅ Done! Files extracted to: ModelNet40


In [29]:
import open3d as o3d

# Load a real object mesh (chair from ModelNet40)
mesh = o3d.io.read_triangle_mesh("ModelNet40/ModelNet40/chair/train/chair_0001.off")
mesh.compute_vertex_normals()

# Normalize: center & scale to unit size
center = mesh.get_center()
mesh.translate(-center)
bbox = mesh.get_axis_aligned_bounding_box()
scale = 1.0 / max(bbox.get_extent())
mesh.scale(scale, center=(0, 0, 0))


TriangleMesh with 2382 points and 2234 triangles.

In [41]:
import numpy as np

def fibonacci_sphere(samples=64, radius=1.8):
    points = []
    phi = np.pi * (3. - np.sqrt(5.))
    for i in range(samples):
        y = 1 - (i / float(samples - 1)) * 2
        r = np.sqrt(1 - y * y)
        theta = phi * i
        x = np.cos(theta) * r
        z = np.sin(theta) * r
        points.append([x * radius, y * radius, z * radius])
    return np.array(points)

candidate_views = fibonacci_sphere(samples=32)


In [47]:
import numpy as np

def generate_normal_based_views(mesh, num_views=64, distance=0.3):
    pcd = mesh.sample_points_uniformly(number_of_points=num_views)
    pcd.estimate_normals()
    points = np.asarray(pcd.points)
    normals = np.asarray(pcd.normals)
    candidate_views = points - normals * distance  # place cameras outside, looking in
    return candidate_views

candidate_views = generate_normal_based_views(mesh, num_views=64, distance=0.3)


In [48]:
from tqdm import tqdm
import open3d as o3d

pcd = mesh.sample_points_uniformly(number_of_points=3000)
view_coverage = []

for view in tqdm(candidate_views, desc="Simulating visibility"):
    _, pt_map = pcd.hidden_point_removal(view, radius=0.5)  # use tight radius for close views
    view_coverage.append(set(pt_map))

total_points = set(range(len(pcd.points)))


Simulating visibility: 100%|██████████████████████████████████████████████████████████| 64/64 [00:00<00:00, 377.65it/s]


In [49]:
from mealpy import FloatVar, SMA

num_views_to_select = 30  # or try 20, 25, etc.

def objective_function(solution):
    indices = [int(round(x)) % len(view_coverage) for x in solution]
    indices = list(set(indices))
    covered = set()
    for i in indices:
        covered |= view_coverage[i]
    uncovered = len(total_points - covered)
    return uncovered

problem_dict = {
    "obj_func": objective_function,
    "minmax": "min",
    "bounds": FloatVar(
        lb=(0,) * num_views_to_select,
        ub=(len(view_coverage) - 1,) * num_views_to_select,
        name="view_index"
    )
}

model = SMA.OriginalSMA(epoch=50, pop_size=20, p_t=0.03)
g_best = model.solve(problem_dict)

selected_indices = sorted(set(int(round(x)) % len(view_coverage) for x in g_best.solution))
covered = set().union(*[view_coverage[i] for i in selected_indices])
coverage_percent = len(covered) / len(total_points) * 100

print(f"\n📌 Selected {len(selected_indices)} viewpoints: {selected_indices}")
print(f"✅ Coverage: {coverage_percent:.2f}%")


2025/07/14 09:01:44 PM, INFO, mealpy.bio_based.SMA.OriginalSMA: OriginalSMA(epoch=50, pop_size=20, p_t=0.03)
2025/07/14 09:01:44 PM, INFO, mealpy.bio_based.SMA.OriginalSMA: >>>Problem: P, Epoch: 1, Current best: 531.0, Global best: 531.0, Runtime: 0.04591 seconds
2025/07/14 09:01:44 PM, INFO, mealpy.bio_based.SMA.OriginalSMA: >>>Problem: P, Epoch: 2, Current best: 635.0, Global best: 531.0, Runtime: 0.03312 seconds
2025/07/14 09:01:44 PM, INFO, mealpy.bio_based.SMA.OriginalSMA: >>>Problem: P, Epoch: 3, Current best: 598.0, Global best: 531.0, Runtime: 0.03571 seconds
2025/07/14 09:01:44 PM, INFO, mealpy.bio_based.SMA.OriginalSMA: >>>Problem: P, Epoch: 4, Current best: 568.0, Global best: 531.0, Runtime: 0.03157 seconds
2025/07/14 09:01:44 PM, INFO, mealpy.bio_based.SMA.OriginalSMA: >>>Problem: P, Epoch: 5, Current best: 584.0, Global best: 531.0, Runtime: 0.03845 seconds
2025/07/14 09:01:44 PM, INFO, mealpy.bio_based.SMA.OriginalSMA: >>>Problem: P, Epoch: 6, Current best: 481.0, Global


📌 Selected 28 viewpoints: [0, 1, 3, 5, 7, 9, 10, 14, 18, 22, 23, 27, 29, 31, 33, 34, 35, 36, 37, 38, 40, 44, 47, 53, 57, 59, 60, 63]
✅ Coverage: 85.13%


In [ ]:
# Visualize mesh + red = selected, gray = others
spheres = []
for i, pos in enumerate(candidate_views):
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.05)
    sphere.translate(pos)
    if i in selected_indices:
        sphere.paint_uniform_color([1, 0, 0])  # red
    else:
        sphere.paint_uniform_color([0.7, 0.7, 0.7])  # gray
    spheres.append(sphere)

o3d.visualization.draw_geometries([mesh] + spheres)
